In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [5]:
mail = pd.read_csv('mail.csv', index_col=None)

In [8]:
mail.head()

,Unnamed: 0,labels,clean
0,0,0,"['', 'date', 'wed', '21', 'aug', '2002', '10',..."
1,1,0,"['', 'martin', 'posted', 'ntassos', 'papadopou..."
2,2,0,"['', 'man', 'threatens', 'explosion', 'moscow'..."
3,3,0,"['', 'klez', 'virus', 'die', 'nalready', 'prol..."
4,4,0,"['', 'wed', 'aug', '21', '2002', '15', '46', '..."


In [9]:
mail.drop('Unnamed: 0', axis=1, inplace =True)

In [10]:
mail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  3051 non-null   int64 
 1   clean   3051 non-null   object
dtypes: int64(1), object(1)
memory usage: 47.8+ KB


In [11]:
mail.describe()

,labels
count,3051.000000
mean,0.163881
std,0.370228
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


### Clean Text with stemming/Lemmatizing

In [12]:
wn = WordNetLemmatizer()

In [13]:
stopword = stopwords.words('english')

In [14]:
mail['clean1'] = mail['clean'].apply(lambda x: " ".join([wn.lemmatize(i).lower() for i in re.sub('[^a-zA-Z0-9]',' ',x).split() if i not in stopword]))

In [15]:
mail

,labels,clean,clean1
0,0,"['', 'date', 'wed', '21', 'aug', '2002', '10',...",date wed 21 aug 2002 10 54 46 0500 chris garri...
1,0,"['', 'martin', 'posted', 'ntassos', 'papadopou...",martin posted ntassos papadopoulos greek sculp...
2,0,"['', 'man', 'threatens', 'explosion', 'moscow'...",man threatens explosion moscow nthursday augus...
3,0,"['', 'klez', 'virus', 'die', 'nalready', 'prol...",klez virus die nalready prolific virus ever kl...
4,0,"['', 'wed', 'aug', '21', '2002', '15', '46', '...",wed aug 21 2002 15 46 ulises ponce wrote hi co...
...,...,...,...
3046,1,"['', 'stumbling', 'nthe', 'greatest', 'way', '...",stumbling nthe greatest way marketing century ...
3047,1,"['', 'mean', 'made', 'usa', 'cna', 'hitting', ...",mean made usa cna hitting road tell ingenuity ...
3048,1,"['', 'html', 'head', 'meta', 'http', 'equiv', ...",html head meta http equiv content language con...
3049,1,"['', 'html', 'body', 'tr', 'valign', 'top', 't...",html body tr valign top td height 295 bgcolor ...


In [17]:
X_train, X_test, y_train, y_test = train_test_split(mail['clean1'],mail.labels,test_size=0.2, random_state=42)

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2440,)
(2440,)
(611,)
(611,)


In [19]:
nb_pipeline = Pipeline([('countvect', CountVectorizer(lowercase=True)),
                       ('mnb', MultinomialNB(alpha=0.0,class_prior=[0.4, 0.6]))])

In [20]:
scores = cross_val_score(nb_pipeline, X_train, y_train, cv=5)
print(scores)

/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


[0.96311475 0.9795082  0.97131148 0.98360656 0.97336066]


/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [21]:
model = nb_pipeline.fit(X_train, y_train)

/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [22]:
y_pred = model.predict(X_test)

In [23]:
model.score(X_test, y_test)

0.983633387888707

In [24]:
metrics.accuracy_score(y_test, y_pred)

0.983633387888707

In [25]:
metrics.confusion_matrix(y_test, y_pred, labels=[0,1])

array([[516,   0],
       [ 10,  85]])

In [26]:
metrics.f1_score(y_test, y_pred, average=None)

array([0.99040307, 0.94444444])

In [27]:
metrics.f1_score(y_test, y_pred, average='macro')

0.9674237577308594

In [28]:
metrics.f1_score(y_test, y_pred, average='micro')

0.983633387888707

In [29]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.9832572943815644

In [30]:
X_test[y_test > y_pred]

2679    result feedback form submitted jamie msn com t...
3050    uncommon exotic pleasure botanical nfeeling ma...
2859    3672cxjk2 471rlyh4856dcwl9 357kcvy4323scva9 71...
2829    xc2 x93connecting business world wide web xc2 ...
2752    greeting nyou receiving letter expressed inter...
2783    dear sir nmy name dr steven duba son mr theo n...
2931    love nworld capital group group funding source...
2784    hi job seeker nwhen create free net temp accou...
2805    nsent mail message nfrom enenkio webtv net rob...
2921    nhallo ni found email id directoric ni russian...
Name: clean1, dtype: object

In [31]:
X_test[y_test < y_pred]

Series([], Name: clean1, dtype: object)

In [32]:
nb_pipeline_tf = Pipeline([('tfidfvect', TfidfVectorizer()),
                       ('mnb', MultinomialNB(alpha=0.0,class_prior=[0.4, 0.6]))])

In [33]:
scores = cross_val_score(nb_pipeline_tf, X_train, y_train, cv=5)
print(scores)

/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in

[0.97131148 0.98770492 0.9795082  0.98565574 0.97745902]


In [34]:
tfmodel = nb_pipeline_tf.fit(X_train, y_train)

/Users/tupran/opt/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [35]:
y_pred_tf = tfmodel.predict(X_test)

In [36]:
metrics.accuracy_score(y_test, y_pred_tf)

0.9852700490998363

In [37]:
metrics.confusion_matrix(y_test, y_pred_tf, labels=[0,1])

array([[516,   0],
       [  9,  86]])

In [38]:
metrics.f1_score(y_test, y_pred, average=None)

array([0.99040307, 0.94444444])

In [39]:
metrics.f1_score(y_test, y_pred, average='micro')

0.983633387888707

In [40]:
metrics.f1_score(y_test, y_pred, average='macro')

0.9674237577308594

In [41]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.9832572943815644

In [42]:
X_test[y_test > y_pred_tf]

2679    result feedback form submitted jamie msn com t...
3050    uncommon exotic pleasure botanical nfeeling ma...
2859    3672cxjk2 471rlyh4856dcwl9 357kcvy4323scva9 71...
2829    xc2 x93connecting business world wide web xc2 ...
2783    dear sir nmy name dr steven duba son mr theo n...
2931    love nworld capital group group funding source...
2784    hi job seeker nwhen create free net temp accou...
2805    nsent mail message nfrom enenkio webtv net rob...
2921    nhallo ni found email id directoric ni russian...
Name: clean1, dtype: object

In [43]:
X_test[y_test < y_pred_tf]

Series([], Name: clean1, dtype: object)

In [44]:
test = ['Get started like a pro | Ultimaker Get started like a pro Having trouble viewing this email? Click here . Hi Shailesh, More and more companies are becoming convinced that 3D printing  plays a key role in their digital transformation roadmap. But what would life at  look like if you adopted 3D printing? Where would you and your team start? To give you an idea, check out our 5 top tips to ramp-up 3D printing production. Read the blog Ultimaker - Corporate Headquarters Stationsplein 32, 3511 ED Utrecht, The Netherlands Email: info@ultimaker.com Phone: +31 88 383 4000 If youâd rather not receive these updates,']

In [45]:
nb_pipeline.named_steps['countvect'].transform(test)

<1x33994 sparse matrix of type '<class 'numpy.int64'>'
	with 46 stored elements in Compressed Sparse Row format>

In [46]:
nb_pipeline.predict(test)

array([0])

In [48]:
andro = pd.read_csv('Andro.csv')

In [49]:
andro = andro.T

In [50]:
test = andro[0]

In [51]:
nb_pipeline.named_steps['countvect'].transform(test)

<24x33994 sparse matrix of type '<class 'numpy.int64'>'
	with 745 stored elements in Compressed Sparse Row format>

In [52]:
nb_pipeline.predict(test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0])

In [53]:
nb_pipeline_tf.named_steps['tfidfvect'].transform(test)

<24x33994 sparse matrix of type '<class 'numpy.float64'>'
	with 745 stored elements in Compressed Sparse Row format>

In [59]:
nb_pipeline_tf.predict(test)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0])

In [60]:
test[3]

'WizIQ Email Hi Tupran Shailesh Kumar, You recently changed your WizIQ password. As a security precaution, this notification has been sent to all email addresses associated with your account. For any queries please contact us at: Email: courses@wiziq.com Phone: + 91-172-5020178 Twitter: http://twitter.com/!/wiziqcourses Facebook: http://facebook.com/wiziqcourses Regards, The WizIQ Team -------------------------------------------------------------- DO NOT REPLY TO THIS MAIL --------------------------------------------------------------'

In [62]:
test[21]

'b\'\\n\\nManage Big Data in a Smart Way with Hadoop\\n\\n View this message in your browser\\n<http://mailer.wiziqcourses.com/lists/lt.php?id=ME8HCQUDTVAAABpSAQcKAA9b>\\n\\n <http://mailer.wiziqcourses.com/lists/lt.php?id=ME8ECE4EBlRLBlFSBQIBDQ>\\n\\n <http://mailer.wiziqcourses.com/lists/lt.php?id=ME8FRAEABh8GAFdQAg4A>\\n\\n <http://mailer.wiziqcourses.com/lists/lt.php?id=ME8HCQUCTVAAABpSAQcKAA9b>\\n\\n Hi,\\n\\n Do you know that big enterprises like Facebook and Amazon use Hadoop\\nto manage their huge databases? And many more organizations are\\nlooking to implement it with the help of trained and qualified Hadoop\\nprofessionals.\\n\\n If you are a Database developer/administrator, Business Analyst, Java\\nprogrammer, or an IT professional, passionate about making a rewarding\\ncareer in Big Data, then enroll in the Hadoop Big Data Training course\\non WizIQ today! \\n\\n According to Mckinsey, "The United States alone faces a shortage of\\n140,000 to 190,000 people with analytica